In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [ ]:
# Load the model from the .h5 file
model = load_model('model_NN_final.h5')

In [ ]:
file_path4 = 'data_parsing/Parsed_Data2.csv'
data2 = pd.read_csv(file_path4)

In [ ]:
# Apply the function to each row
data2['specific_sequence'] = data2.apply(extract_specific_sequence, axis=1)


data2['1st_pos'] = data2['specific_sequence'].apply(lambda x: x[0])
data2['2nd_pos'] = data2['specific_sequence'].apply(lambda x: x[1])
data2['3rd_pos'] = data2['specific_sequence'].apply(lambda x: x[2])
data2['4th_pos'] = data2['specific_sequence'].apply(lambda x: x[3])
data2['5th_pos'] = data2['specific_sequence'].apply(lambda x: x[4])

In [ ]:
seq_col = [col for col in data2.columns.tolist() if col.startswith('sequence_')]
data2.drop(seq_col, axis=1, inplace=True)
data2.drop('specific_sequence', axis=1, inplace=True)
data2['1st_pos'] = data2['1st_pos'].astype('category')
data2['2nd_pos'] = data2['2nd_pos'].astype('category')
data2['3rd_pos'] = data2['3rd_pos'].astype('category')
data2['4th_pos'] = data2['4th_pos'].astype('category')
data2['5th_pos'] = data2['5th_pos'].astype('category')

In [ ]:
grouped_data2 = data2.groupby("transcript_id")

# Apply the standardization function to the "0 mean" column within each group
data2["0 mean standardized"] = grouped_data2["0 mean"].transform(standardize_column_mean)
data2["0 sd standardized"] = grouped_data2["0 sd"].transform(standardize_column_sd)
data2["0 length standardized"] = grouped_data2["0 length"].transform(standardize_column_mean)

In [ ]:
final_predict_dataset2 = data2.drop(columns=['transcript_id','position','0 length', '0 sd', '0 mean', '0 min', '0 max'])

# Perform one-hot encoding for categorical columns
final_predict_dataset2 = pd.get_dummies(final_predict_dataset2, columns=['1st_pos', '2nd_pos', '3rd_pos', '4th_pos', '5th_pos'])


In [ ]:
transcript2 = data2[['transcript_id','position']]
data2_pred_prob = model.predict(final_predict_dataset2)

In [ ]:
data2_pred_p = pd.DataFrame(pd.DataFrame(data2_pred_prob))
data2_pred_p.columns = ['score']
result_data2 = pd.merge(transcript2, data2_pred_p, left_index=True, right_index=True, how='inner')
agg_functions = {'score': 'mean'}

result_data2 = result_data2.groupby(['transcript_id', 'position']).agg(agg_functions).reset_index()

In [ ]:
result_data2.to_csv('prediction_generation/Final_Data2_Result.csv', index=False)